In [1]:
#!pip install textblob

In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from textblob import TextBlob
import re
import json
import xgboost as xgb

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import preprocessing
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB 
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler

#Metrics
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report

In [3]:
#import library
!pip install contractions 

import pandas as pd
import numpy as np

import nltk
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import unicodedata
import contractions
from contractions import contractions_dict
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize, regexp_tokenize 
from nltk.stem import PorterStemmer, WordNetLemmatizer
import re


import re, string, unicodedata
import nltk
import contractions
from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import WordNetLemmatizer


[notice] A new release of pip available: 22.3 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\edcyh\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\edcyh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\edcyh\AppData\Roaming\nltk_data...


[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\edcyh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
data_list = []

with open('emotion text/data.jsonl', 'r') as f:
    data_str = f.read()
    data_str = data_str.replace('\n', ',')  # add commas between objects
    data_str = '[' + data_str[:-1] + ']'   # enclose the entire string in brackets to create a valid JSON array
    data_list = json.loads(data_str)

# Convert the list of JSON objects into a DataFrame
df = pd.DataFrame(data_list)

# Print the first few rows of the DataFrame
print(df.head())


                                                text  label
0  i feel awful about it too because it s my job ...      0
1                              im alone i feel awful      0
2  ive probably mentioned this before but i reall...      1
3           i was feeling a little low few days back      0
4  i beleive that i am much more sensitive to oth...      2


In [5]:
df = df[df['label'].isin([2, 5]) == False]

In [6]:
df.replace({4: 2}, inplace=True)

In [7]:
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text



# Define function to expand contractions
def expand_contractions(phrase):
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase


# special_characters removal
def remove_special_characters(text, remove_digits=True):
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    return text


def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words


def remove_punctuation_and_splchars(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_word = remove_special_characters(new_word, True)
            new_words.append(new_word)
    return new_words

def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

stopword_list= stopwords.words('english')
stopword_list.append('im')
stopword_list.append('ive')
new_words = ["not", "no", "never", "neither", "nor", "none"]
for word in new_words:
    if word in stopword_list:
        stopword_list.remove(word)

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopword_list:
            new_words.append(word)
    return new_words

def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

def normalize(words):
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation_and_splchars(words)
    words = remove_stopwords(words)
    return words

def lemmatize(words):
    lemmas = lemmatize_verbs(words)
    return lemmas

def normalize_and_lemmaize(input):
    sample = denoise_text(input)
    sample = remove_special_characters(sample)
    words = nltk.word_tokenize(sample)
    words = normalize(words)
    lemmas = lemmatize(words)
    return ' '.join(lemmas)

In [8]:
df['clean_data'] = df['text'].map(lambda text: normalize_and_lemmaize(text))

### LTSM

In [9]:
df

,text,label,clean_data
0,i feel awful about it too because it s my job ...,0,feel awful job get position succeed happen
1,im alone i feel awful,0,alone feel awful
2,ive probably mentioned this before but i reall...,1,probably mention really feel proud actually ke...
3,i was feeling a little low few days back,0,feel little low days back
6,i am one of those people who feels like going ...,1,one people feel like go gym worthwhile hour
...,...,...,...
416804,that was what i felt when i was finally accept...,1,felt finally accept bulgarian conservatorie ap...
416805,i take every day as it comes i m just focussin...,2,take every day come focus eat better moment no...
416806,i just suddenly feel that everything was fake,0,suddenly feel everything fake
416807,im feeling more eager than ever to claw back w...,1,feel eager ever claw back go pear shape last w...


In [10]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelBinarizer

# Read in the cleaned text data and labels
X = df['clean_data']
y = df['label']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the tokenizer with a maximum vocabulary size of 5000 words
tokenizer = Tokenizer(num_words=10000)

# Fit the tokenizer on the training data
tokenizer.fit_on_texts(X_train)

# Convert the text data to numerical sequences
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad the sequences to a maximum length of 100
max_seq_length = 100
X_train_pad = pad_sequences(X_train_seq, maxlen=max_seq_length)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_seq_length)

# Convert the label data to a matrix using LabelBinarizer
lb = LabelBinarizer()
y_train = lb.fit_transform(y_train)
y_test = lb.transform(y_test)


In [11]:
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Build the LSTM model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=128, input_length=max_seq_length))
model.add(LSTM(128))
model.add(Dense(1024, activation='relu'))
model.add(Dense(4, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
history = model.fit(X_train_pad, y_train, validation_data=(X_test_pad, y_test), epochs=5, batch_size=32, 
                    callbacks=[EarlyStopping(patience=2, restore_best_weights=True)])

# Evaluate the model
loss, accuracy = model.evaluate(X_test_pad, y_test)
print('Test accuracy:', accuracy)

# Save the model to a file
#model.save('NLP_model.h5')

Epoch 1/5
9183/9183 [==============================] - 1094s 119ms/step - loss: 0.0992 - accuracy: 0.9555 - val_loss: 0.0574 - val_accuracy: 0.9707
Epoch 2/5
9183/9183 [==============================] - 1106s 120ms/step - loss: 0.0523 - accuracy: 0.9721 - val_loss: 0.0573 - val_accuracy: 0.9695
Epoch 3/5
9183/9183 [==============================] - 1241s 135ms/step - loss: 0.0457 - accuracy: 0.9742 - val_loss: 0.0545 - val_accuracy: 0.9717
Epoch 4/5
9183/9183 [==============================] - 1135s 124ms/step - loss: 0.0402 - accuracy: 0.9761 - val_loss: 0.0626 - val_accuracy: 0.9709
Epoch 5/5
2296/2296 [==============================] - 68s 30ms/step - loss: 0.0545 - accuracy: 0.9717
Test accuracy: 0.9717385768890381


In [39]:
model.save('NLP_model.h5')

In [40]:
from tensorflow import keras
new_model = keras.models.load_model('NLP_model.h5')
new_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 128)          6919040   
                                                                 
 lstm (LSTM)                 (None, 128)               131584    
                                                                 
 dense (Dense)               (None, 1024)              132096    
                                                                 
 dense_1 (Dense)             (None, 4)                 4100      
                                                                 
Total params: 7,186,820
Trainable params: 7,186,820
Non-trainable params: 0
_________________________________________________________________


In [46]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelBinarizer

# Define the custom text to test
custom_text = "Yes! This model is working"
print(normalize_and_lemmaize(custom_text))

# Tokenize the custom text
custom_text_seq = tokenizer.texts_to_sequences([normalize_and_lemmaize(custom_text)])

# Pad the sequence to the same length as used for the training data
custom_text_pad = pad_sequences(custom_text_seq, maxlen=max_seq_length)

# Make predictions using the trained model
predictions = new_model.predict(custom_text_pad)

# Convert the predictions to class labels using inverse_transform
predicted_labels = lb.inverse_transform(predictions)

print('Predicted label:', predicted_labels[0])
print("\n0:Sad|1:Happy|2:Fear|3:Anger")

yes model work
1/1 [==============================] - 0s 20ms/step
Predicted label: 1

0:Sad|1:Happy|2:Fear|3:Anger
